In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
#实现多输出
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab  
import matplotlib.pyplot as plt  
import textdistance
import random
import re

In [2]:
sanction_prim = pd.read_csv('cons_prim.csv',header=None,names=['prim_id','PRIMARY_NAME'],usecols=[0,1]).set_index('prim_id').dropna()
sanction_alt = pd.read_csv('cons_alt.csv',header=None,names=['alt_id','ALT_NAME'],usecols=[0,3]).set_index('alt_id').dropna()
sanction_match = pd.merge(sanction_prim,sanction_alt,how='inner',left_index=True,right_index=True)
sanction_match.index = range(len(sanction_match))
sanction_match['State'] = 'Match'
sanction_match

,PRIMARY_NAME,ALT_NAME,State
0,BANK OF KUNLUN CO LTD,KARAMAY URBAN CREDIT COOPERATIVES,Match
1,BANK OF KUNLUN CO LTD,KARAMAY CITY COMMERCIAL BANK CO LTD.,Match
2,VTB BANK OAO,JSC VTB BANK,Match
3,VTB BANK OAO,"BANK VTB, OPEN JOINT-STOCK COMPANY",Match
4,VTB BANK OAO,VNESHTORGBANK,Match
...,...,...,...
925,"UDWAN, Atef","ADWAN, Atef Ibrahim",Match
926,"UDWAN, Atef","IDWAN, Atef Ibrahim Mohammad",Match
927,"AL-SHARAFI, Yousef","AL-SHRAFI, Yousef Awad",Match
928,"AL-ASHQAR, Ismail","AL-ALASHQAR, Ismail Abdallatif",Match


In [3]:
sanction_not_match = pd.DataFrame(columns=('PRIMARY_NAME', 'ALT_NAME', 'State'))
for i in range(len(sanction_prim)):
    for j in range(i,i+10):
        if(sanction_prim.iloc[i].name!=sanction_alt.iloc[j].name):
            sanction_not_match = sanction_not_match.append({'PRIMARY_NAME':sanction_prim.iloc[i]['PRIMARY_NAME'],'ALT_NAME':sanction_alt.iloc[j]['ALT_NAME'],'State':'Not Match'},ignore_index=True)
sanction_not_match

,PRIMARY_NAME,ALT_NAME,State
0,"HANIYA, Ismail Abdul Salah","ABU TAIR, Mohammed Mahmud",Not Match
1,"HANIYA, Ismail Abdul Salah","ABOU TAYR, Mohammad Mahmoud",Not Match
2,"HANIYA, Ismail Abdul Salah","EL-SHANTI, Jameela Abdallah",Not Match
3,"HANIYA, Ismail Abdul Salah","AL SHANTEE, Jamila Abed Allah",Not Match
4,"HANIYA, Ismail Abdul Salah","ALA AL DIN, Mohamad Nouman",Not Match
...,...,...,...
4058,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),JOINT STOCK COMPANY RT-CHEMICAL TECHNOLOGIES A...,Not Match
4059,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),"RT-KHIMKOMPOZIT, AO",Not Match
4060,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),RT-KHIMKOMPOZIT OAO,Not Match
4061,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),RT-KHIMKOMPOZIT OJSC,Not Match


In [4]:
data = pd.read_excel('NameMatchingTrainingData.xlsx')
data = data.append(sanction_match,ignore_index=True).append(sanction_not_match,ignore_index=True)
data

,PRIMARY_NAME,ALT_NAME,State
0,GAZPROMBANK UPRAVLENIE AKTIVAMI,GAZPROMBANK ASSET MANAGEMENT ZAO,Match
1,GAZPROMBANK UPRAVLENIE AKTIVAMI,CLOSED JOINT-STOCK COMPANY GAZPROMBANK-UPRAVLE...,Match
2,UNITED INSTRUMENT MANUFACTURING CORPORATION,JSC-UNITED-INSTRUMENT-MANUFACTURING-CORPORATION,Match
3,"GAZ-OIL, OOO",GAZ-OIL,Match
4,Mr. Ken Wong,KEN WONG KAW,Match
...,...,...,...
41857,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),JOINT STOCK COMPANY RT-CHEMICAL TECHNOLOGIES A...,Not Match
41858,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),"RT-KHIMKOMPOZIT, AO",Not Match
41859,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),RT-KHIMKOMPOZIT OAO,Not Match
41860,CHINA SHIPBUILDING INDUSTRY CORPORATION (CSIC),RT-KHIMKOMPOZIT OJSC,Not Match


In [5]:
def clean_data(data):
    import re
    regex = re.compile(u"\\(.*?\\)|\\{.*?}|\\[.*?]")
    for col in data:
        data[col] = data[col].str.lower()
        data[col] =data[col].str.replace('mr.','')
        data[col] =data[col].str.replace('mrs.','')
        data[col] =data[col].str.replace('\n',' \n')
        data[col] =data[col].str.replace('.',' ')
        data[col] =data[col].str.replace(',',' ')
        data[col] =data[col].str.replace('\'','')
        data[col] =data[col].str.replace(' & ',' and ')
        data[col] =data[col].str.replace(' &',' and ')
        data[col] =data[col].str.replace('& ',' and ')
        data[col] =data[col].str.replace('&',' and ')
        data[col] =data[col].str.replace(' l p ',' lp ')
        data[col] =data[col].str.replace('l l p ','llp ')
        data[col] =data[col].str.replace('l l c ','llc ')
        data[col] =data[col].str.replace(' ltd ',' limited ')
        data[col] =data[col].str.replace(' p l c ',' plc')
        data[col] =data[col].str.replace(' co ltd ',' company limited ')
        data[col] =data[col].str.replace(' co ',' company ')
        data[col] =data[col].str.replace(' corp ',' corporation ')
        data[col] =data[col].str.replace(' univ ',' university ')
        data[col] =data[col].str.replace(' fdtn ',' foundation ')
        data[col] =data[col].str.replace(' inc ',' incorporated ')
        data[col] =data[col].str.replace(' assoc ',' association ')
        data[col] =data[col].str.replace(' mgmt ',' management ')
        data[col] =data[col].str.replace(' svsc ',' services ')
        data[col] =data[col].str.replace(' ag ',' aktiengesellschaft ')
        data[col] =data[col].str.replace(' govt ',' goverment ')
        data[col] =data[col].str.replace(' svgs ',' savings ')
        data[col] =data[col].str.replace(' intl ',' international ')
        data[col] =data[col].str.replace('intl ','international ')
        data[col] =data[col].str.replace(' investmentco lt ',' investment company limited ')
        data[col] =data[col].str.replace(regex,'')
        data[col] = data[col].str.strip()
        data[col] = data[col].str.replace('\s+',' ')
    return data
data = clean_data(data)
data

<ipython-input-5-d8e431bac43c>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] =data[col].str.replace('mr.','')
<ipython-input-5-d8e431bac43c>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] =data[col].str.replace('mrs.','')
<ipython-input-5-d8e431bac43c>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data[col] =data[col].str.replace('.',' ')
<ipython-input-5-d8e431bac43c>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace('\s+',' ')


,PRIMARY_NAME,ALT_NAME,State
0,gazprombank upravlenie aktivami,gazprombank asset management zao,match
1,gazprombank upravlenie aktivami,closed joint-stock company gazprombank-upravle...,match
2,united instrument manufacturing corporation,jsc-united-instrument-manufacturing-corporation,match
3,gaz-oil ooo,gaz-oil,match
4,ken wong,ken wong kaw,match
...,...,...,...
41857,china shipbuilding industry corporation,joint stock company rt-chemical technologies a...,not match
41858,china shipbuilding industry corporation,rt-khimkompozit ao,not match
41859,china shipbuilding industry corporation,rt-khimkompozit oao,not match
41860,china shipbuilding industry corporation,rt-khimkompozit ojsc,not match


In [6]:
data.to_excel('cleaned.xlsx')

In [7]:
def InStringSerch(a,b):
    return int(a in b or b in a)

In [8]:
data['Cosine Similarity'] = [textdistance.cosine.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Levenshtein Distance'] = [textdistance.levenshtein.distance(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Jaro-Winkler Distance'] = [textdistance.jaro_winkler.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Jaccard Index'] = [textdistance.jaccard.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Monge Elkan'] = [textdistance.monge_elkan.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['MRA'] = [textdistance.mra.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Longest Common Substring'] = [textdistance.lcsstr.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['Longest Common Subsequence'] = [textdistance.lcsseq.similarity(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data['In String Search'] = [InStringSerch(str(data.iloc[i]['PRIMARY_NAME']),str(data.iloc[i]['ALT_NAME'])) for i in range(len(data))]
data

,PRIMARY_NAME,ALT_NAME,State,Cosine Similarity,Levenshtein Distance,Jaro-Winkler Distance,Jaccard Index,Monge Elkan,MRA,Longest Common Substring,Longest Common Subsequence,In String Search
0,gazprombank upravlenie aktivami,gazprombank asset management zao,match,0.666751,17,0.847686,0.500000,0.024974,4,12,18,0
1,gazprombank upravlenie aktivami,closed joint-stock company gazprombank-upravle...,match,0.731083,28,0.683537,0.534483,0.032258,3,19,30,0
2,united instrument manufacturing corporation,jsc-united-instrument-manufacturing-corporation,match,0.889768,7,0.802099,0.800000,0.021633,3,13,40,0
3,gaz-oil ooo,gaz-oil,match,0.797724,4,0.927273,0.636364,0.082645,4,7,7,1
4,ken wong,ken wong kaw,match,0.816497,4,0.933333,0.666667,0.125000,3,8,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
41857,china shipbuilding industry corporation,joint stock company rt-chemical technologies a...,not match,0.640807,51,0.597034,0.445946,0.023669,1,3,20,0
41858,china shipbuilding industry corporation,rt-khimkompozit ao,not match,0.452911,32,0.502849,0.266667,0.015122,1,2,8,0
41859,china shipbuilding industry corporation,rt-khimkompozit oao,not match,0.440831,31,0.494077,0.260870,0.015122,1,2,9,0
41860,china shipbuilding industry corporation,rt-khimkompozit ojsc,not match,0.465475,33,0.485470,0.282609,0.016437,0,2,7,0


In [19]:
data.to_excel('cleaned_new.xlsx')